In [1]:
import torch
import ChatTTS
from  IPython.lib.display import Audio 

In [2]:
chat = ChatTTS.Chat()

In [3]:
chat.load_models(compile=True) # 设置为Flase获得更快速度，设置为True获得更佳效果

INFO:ChatTTS.core:Load from cache: /root/.cache/huggingface/hub/models--2Noise--ChatTTS/snapshots/cc14302f34d7855eb3420d1fd48345012ff1460d
INFO:ChatTTS.core:use cuda:0
INFO:ChatTTS.core:vocos loaded.
INFO:ChatTTS.core:dvae loaded.
INFO:ChatTTS.core:gpt loaded.
INFO:ChatTTS.core:decoder loaded.
INFO:ChatTTS.core:tokenizer loaded.
INFO:ChatTTS.core:All initialized.


In [4]:
# speaker = torch.load('speaker/speaker_5_girl.pth')

In [5]:
def tts(text, oral=3, laugh=3, bk=3):

    '''
    输入文本，输出音频
    '''
    rand_spk = chat.sample_random_speaker()
    # 句子全局设置：讲话人音色和速度
    params_infer_code = {
        'spk_emb': rand_spk,
    }

    # 句子全局设置：口语连接、笑声、停顿程度
    # oral：连接词，AI可能会自己加字，取值范围 0-9，比如：卡壳、嘴瓢、嗯、啊、就是之类的词。不宜调的过高。
    # laugh：笑，取值范围 0-9
    # break：停顿，取值范围 0-9
    params_refine_text = {
        'prompt': '[oral_{}][laugh_{}][break_{}]'.format(oral, laugh, bk)
    }

    wavs = chat.infer(text, params_refine_text=params_refine_text, params_infer_code=params_infer_code)

    return wavs

In [6]:
test = '一个美国人和一个苏联人在讨论他们国家的自由。美国人说：在美国，我们有绝对的言论自由。我可以走到白宫前面，喊打倒里根！而不会有任何问题。苏联人回应道：这没什么特别的。在苏联，我们也有绝对的言论自由。我也可以走到红场，喊打倒里根！而不会有任何问题。'

In [7]:
wavs = tts(test, oral=2, laugh=5, bk=3)

INFO:ChatTTS.core:All initialized.
  0%|          | 0/384 [00:00<?, ?it/s]/root/miniconda3/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/root/miniconda3/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/root/miniconda3/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/root/miniconda3/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
/root/miniconda3/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:135: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_pre

In [8]:
Audio(wavs[0], rate=24000)

In [9]:
wavs[0]

array([[ 2.6628224e-04, -6.5525220e-04, -2.2353674e-03, ...,
        -1.4251129e-04, -2.7774775e-04,  1.7815590e-05]], dtype=float32)

In [12]:
def vec2wav(pcm_vec, wav_file, framerate = 16000):
    """ 
    将numpy数组转为单通道wav文件
    :param pcm_vec: 输入的numpy向量
    :param wav_file: wav文件名
    :param framerate: 采样率
    :return:
    """
    import wave

   # pcm_vec = np.clip(pcm_vec, -32768, 32768)
   
    # if np.max(np.abs(pcm_vec)) > 1.0:
    #     pcm_vec *= 32767 / max(0.01, np.max(np.abs(pcm_vec)))
    # else:
    pcm_vec = pcm_vec * 32768
    pcm_vec = pcm_vec.astype(np.int16)
    wave_out = wave.open(wav_file, 'wb')
    wave_out.setnchannels(1)
    wave_out.setsampwidth(2)
    wave_out.setframerate(framerate)
    wave_out.writeframes(pcm_vec)


In [14]:
import numpy as np

In [17]:
vec2wav(wavs[0],"output/test2.wav",24000)